In [1]:
%%capture --no-stderr
!pip install phidata pypdf arxiv serpapi composio_phidata

In [2]:
import os
from google.colab import userdata

In [3]:
from phi.agent import Agent, RunResponse
from phi.model.openai import OpenAIChat
from composio_phidata import Action, ComposioToolSet
from phi.tools.arxiv_toolkit import ArxivToolkit
from phi.utils.pprint import pprint_run_response
from phi.tools.serpapi_tools import SerpApiTools
import os

# Collect API keys from the user
openai_api_key = userdata.get('OPENAI_API_KEY')
composio_api_key = userdata.get('COMPOSIO_API_KEY')
serpapi_api_key = userdata.get('SERPER_API_KEY')
# Validate API keys
if not openai_api_key or not composio_api_key or not serpapi_api_key:
    print("Error: Please provide all required API keys.")
    exit()

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = openai_api_key

try:
    composio_toolset = ComposioToolSet(api_key=composio_api_key)
    google_docs_tool = composio_toolset.get_tools(actions=[Action.GOOGLEDOCS_CREATE_DOCUMENT])[0]
except Exception as e:
    print(f"Error initializing ComposioToolSet: {e}")
    exit()

# Create the agents
professor_agent = Agent(
    name="Professor",
    role="Research and Knowledge Specialist",
    model=OpenAIChat(id="gpt-4o-mini", api_key=openai_api_key),
    tools=[google_docs_tool],
    instructions=[
        "Create a comprehensive knowledge base that covers fundamental concepts, advanced topics, and current developments of the given topic.",
        "Explain the topic from first principles first. Include key terminology, core principles, and practical applications and make it as a detailed report that anyone who's starting out can read and get maximum value out of it.",
        "Make sure it is formatted in a way that is easy to read and understand. DONT FORGET TO CREATE THE GOOGLE DOCUMENT.",
        "Open a new Google Doc and write down the response of the agent neatly with great formatting and structure in it. **Include the Google Doc link in your response.**",
    ],
    show_tool_calls=True,
    markdown=True,
)

academic_advisor_agent = Agent(
    name="Academic Advisor",
    role="Learning Path Designer",
    model=OpenAIChat(id="gpt-4o-mini", api_key=openai_api_key),
    tools=[google_docs_tool],
    instructions=[
        "Using the knowledge base for the given topic, create a detailed learning roadmap.",
        "Break down the topic into logical subtopics and arrange them in order of progression, including estimated time commitments for each section.",
        "Present the roadmap in a clear, structured format. DONT FORGET TO CREATE THE GOOGLE DOCUMENT.",
        "Open a new Google Doc and write down the response of the agent neatly with great formatting and structure in it. **Include the Google Doc link in your response.**",
    ],
    show_tool_calls=True,
    markdown=True,
)

research_librarian_agent = Agent(
    name="Research Librarian",
    role="Learning Resource Specialist",
    model=OpenAIChat(id="gpt-4o-mini", api_key=openai_api_key),
    tools=[google_docs_tool, SerpApiTools(api_key=serpapi_api_key)],
    instructions=[
        "Make a list of high-quality learning resources for the given topic.",
        "Use the SerpApi search tool to find current and relevant learning materials.",
        "Include technical blogs, GitHub repositories, official documentation, video tutorials, and courses.",
        "Present the resources in a curated list with descriptions and quality assessments. DONT FORGET TO CREATE THE GOOGLE DOCUMENT.",
        "Open a new Google Doc and write down the response of the agent neatly with great formatting and structure in it. **Include the Google Doc link in your response.**",
    ],
    show_tool_calls=True,
    markdown=True,
)

teaching_assistant_agent = Agent(
    name="Teaching Assistant",
    role="Exercise Creator",
    model=OpenAIChat(id="gpt-4o-mini", api_key=openai_api_key),
    tools=[google_docs_tool, SerpApiTools(api_key=serpapi_api_key)],
    instructions=[
        "Create comprehensive practice materials for the given topic.",
        "Use the SerpApi search tool to find example problems and real-world applications.",
        "Include progressive exercises, quizzes, hands-on projects, and real-world application scenarios.",
        "Ensure the materials align with the roadmap progression.",
        "Provide detailed solutions and explanations for all practice materials. DONT FORGET TO CREATE THE GOOGLE DOCUMENT.",
        "Open a new Google Doc and write down the response of the agent neatly with great formatting and structure in it. **Include the Google Doc link in your response.**",
    ],
    show_tool_calls=True,
    markdown=True,
)

# Get the topic from the user
topic = input("Enter the topic you want to learn about: ").strip()
if not topic:
    print("Error: Please provide a valid topic.")
    exit()

# Run the agents
print("Generating Knowledge Base...")
professor_response: RunResponse = professor_agent.run(
    f"The topic is: {topic}. Don't forget to add the Google Doc link in your response.", stream=False
)

print("Generating Learning Roadmap...")
academic_advisor_response: RunResponse = academic_advisor_agent.run(
    f"The topic is: {topic}. Don't forget to add the Google Doc link in your response.", stream=False
)

print("Curating Learning Resources...")
research_librarian_response: RunResponse = research_librarian_agent.run(
    f"The topic is: {topic}. Don't forget to add the Google Doc link in your response.", stream=False
)

print("Creating Practice Materials...")
teaching_assistant_response: RunResponse = teaching_assistant_agent.run(
    f"The topic is: {topic}. Don't forget to add the Google Doc link in your response.", stream=False
)

# Display the responses
def display_response(agent_name, response):
    print(f"\n{agent_name} Response:")
    print(response.content)
    pprint_run_response(response, markdown=True)

print("\n--- Google Doc Links ---")
def extract_google_doc_link(response_content):
    if "https://docs.google.com" in response_content:
        return response_content.split("https://docs.google.com")[1].split()[0]
    return None

professor_doc_link = extract_google_doc_link(professor_response.content)
academic_advisor_doc_link = extract_google_doc_link(academic_advisor_response.content)
research_librarian_doc_link = extract_google_doc_link(research_librarian_response.content)
teaching_assistant_doc_link = extract_google_doc_link(teaching_assistant_response.content)

if professor_doc_link:
    print(f"- Professor Document: https://docs.google.com{professor_doc_link}")
if academic_advisor_doc_link:
    print(f"- Academic Advisor Document: https://docs.google.com{academic_advisor_doc_link}")
if research_librarian_doc_link:
    print(f"- Research Librarian Document: https://docs.google.com{research_librarian_doc_link}")
if teaching_assistant_doc_link:
    print(f"- Teaching Assistant Document: https://docs.google.com{teaching_assistant_doc_link}")

# Display detailed responses
display_response("Professor", professor_response)
display_response("Academic Advisor", academic_advisor_response)
display_response("Research Librarian", research_librarian_response)
display_response("Teaching Assistant", teaching_assistant_response)


INFO:composio.utils.shared:Actions cache is outdated, refreshing cache...


Enter the topic you want to learn about: Responsible AI
Generating Knowledge Base...
Generating Learning Roadmap...
Curating Learning Resources...


INFO     Searching Google for: Responsible AI learning resources

INFO     Searching Google for: Responsible AI learning resources

INFO     Searching Google for: Responsible AI resources courses tutorials documentation

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Running:                                                                                                        │
│                                                                                                                 │
│  • function_template(title=Learning Roadmap for Responsible AI, text=...)                                       │
│                                                                                                                 │
│ I have created a detailed learning roadmap for the topic of Responsible AI. You can view and access it through  │
│ the link provided below:                                                                                        │
│                                                                                                                 │
│ ]8;id=13971;https://docs.google.com/document/d/1y2M44wiv8cTDkFGGcUtRXD48JiMKEtnd4hTycHEI5ok/edit?usp=sharing\Learning Roadmap for Responsible AI]8;;\                                                                             │
│                                                                                                                 │
│                                        Overview of the Learning Roadmap                                         │
│                                                                                                                 │
│  • 1. Introduction to AI (3 hours)                                                                              │
│  • 2. The Importance of Responsibility in AI (2 hours)                                                          │
│  • 3. Ethical Frameworks (4 hours)                                                                              │
│  • 4. Bias and Fairness in AI (4 hours)                                                                         │
│  • 5. Transparency and Explainability (3 hours)                                                                 │
│  • 6. Accountability in AI (3 hours)                                                                            │
│  • 7. Security and Privacy Implications (3 hours)                                                               │
│  • 8. Future Trends and Challenges (3 hours)                                                                    │
│                                                                                                                 │
│ Total Estimated Time: 22 hours                                                                                  │
│                                                                                                                 │
│ Feel free to explore the document for more details on each section of the learning roadmap.                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Research Librarian Response:

Running:
 - search_google(query=Responsible AI learning resources, num_results=10)


Running:
 - search_google(query=Responsible AI learning resources, num_results=10)


Running:
 - search_google(query=Responsible AI resources courses tutorials documentation, num_results=10)


Running:
 - search_google(query=Responsible AI resources courses tutorials documentation, num_results=10)
 - search_google(query=Responsible AI learning materials, num_results=10)


Running:
 - search_google(query=Responsible AI resources and tutorials, num_results=10)


Running:
 - function_template(title=Curated Resources for Responsible AI, text=...)

I have curated a list of high-quality learning resources on the topic of Responsible AI. You can find all the details neatly organized in a Google Document. Here's the summary of the resources along with the link:

## Curated List of Resources for Responsible AI

### Online Courses
1. **AI For Everyone by Andrew Ng (Coursera)**  
  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Running:                                                                                                        │
│                                                                                                                 │
│  • search_google(query=Responsible AI learning resources, num_results=10)                                       │
│                                                                                                                 │
│ Running:                                                                                                        │
│                                                                                                                 │
│  • search_google(query=Responsible AI learning resources, num_results=10)                                       │
│                                                                                                                 │
│ Running:                                                                                                        │
│                                                                                                                 │
│  • search_google(query=Responsible AI resources courses tutorials documentation, num_results=10)                │
│                                                                                                                 │
│ Running:                                                                                                        │
│                                                                                                                 │
│  • search_google(query=Responsible AI resources courses tutorials documentation, num_results=10)                │
│  • search_google(query=Responsible AI learning materials, num_results=10)                                       │
│                                                                                                                 │
│ Running:                                                                                                        │
│                                                                                                                 │
│  • search_google(query=Responsible AI resources and tutorials, num_results=10)                                  │
│                                                                                                                 │
│ Running:                                                                                                        │
│                                                                                                                 │
│  • function_template(title=Curated Resources for Responsible AI, text=...)                                      │
│                                                                                                                 │
│ I have curated a list of high-quality learning resources on the topic of Responsible AI. You can find all the   │
│ details neatly organized in a Google Document. Here's the summary of the resources along with the link:         │
│                                                                                                                 │
│                                                                                                                 │
│                                  Curated List of Resources for Responsible AI                                   │
│                                                                                                                 │
│                                                 Online Courses                                                  │
│                                                                                                                 │
│  1 AI For Everyone by Andrew Ng (Coursera)            


Teaching Assistant Response:

Running:
 - search_google(query=Responsible AI practice exercises, num_results=5)


Running:
 - search_google(query=Responsible AI educational resources, num_results=5)


Running:
 - search_google(query=Responsible AI exercises and projects, num_results=5)


Running:
 - function_template(title=Responsible AI Practice Materials, text=...)

I have created a comprehensive document on "Responsible AI Practice Materials" which includes exercises, quizzes, projects, and real-world applications. You can access it at the following link:

[Responsible AI Practice Materials](https://docs.google.com/document/d/1nZiiFcHCp1oBHAyygu6WIj7duvPKlVTF6du9U6E3ws4/edit?usp=sharing)

Here is a brief overview of the content included in the document:

## Responsible AI Practice Materials

### Introduction
Discusses the importance of responsible AI in today's world.

### Exercises
- **Case Study Analysis**: Identify ethical dilemmas and propose solutions based on real-world cases

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Running:                                                                                                        │
│                                                                                                                 │
│  • search_google(query=Responsible AI practice exercises, num_results=5)                                        │
│                                                                                                                 │
│ Running:                                                                                                        │
│                                                                                                                 │
│  • search_google(query=Responsible AI educational resources, num_results=5)                                     │
│                                                                                                                 │
│ Running:                                                                                                        │
│                                                                                                                 │
│  • search_google(query=Responsible AI exercises and projects, num_results=5)                                    │
│                                                                                                                 │
│ Running:                                                                                                        │
│                                                                                                                 │
│  • function_template(title=Responsible AI Practice Materials, text=...)                                         │
│                                                                                                                 │
│ I have created a comprehensive document on "Responsible AI Practice Materials" which includes exercises,        │
│ quizzes, projects, and real-world applications. You can access it at the following link:                        │
│                                                                                                                 │
│ ]8;id=108206;https://docs.google.com/document/d/1nZiiFcHCp1oBHAyygu6WIj7duvPKlVTF6du9U6E3ws4/edit?usp=sharing\Responsible AI Practice Materials]8;;\                                                                               │
│                                                                                                                 │
│ Here is a brief overview of the content included in the document:                                               │
│                                                                                                                 │
│                                                                                                                 │
│                                        Responsible AI Practice Materials                                        │
│                                                                                                                 │
│                                                  Introduction                                                   │
│                                                                                                                 │
│ Discusses the importance of responsible AI in today's world.                                                    │
│                                                                                                                 │
│                                                    Exercises                                                    │
│                                                                                                                 │
│  • Case Study Analysis: Identify ethical dilemmas a